In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB
import pickle
import logging
import folium

In [3]:
poi_df = pd.read_csv("data/CombinedPOI.csv")
cp_df = pd.read_csv("data/CombinedCarpark.csv")

## Create a function to extract any points within the CombinedPOI.csv or CombinedCarpark.csv that are within the chosen area.

The list of residential areas that are given as a choice are:
1. Tampines
2. Bishan
3. Jurong West
4. Jurong East
5. Boon Lay
6. Woodlands
7. Punggol
8. Seng Kang
9. Hougang
10. Queenstown
11. Clementi
12. Orchard Road
13. Downtown Core
14. Marina Bay 

In [9]:
def is_within_residential_area(area_name, data):
    # Define the latitude and longitude ranges for each residential area
    residential_areas = [
        {"name": "Tampines", "latitude": (1.3496, 1.3634), "longitude": (103.9167, 103.9544)},
        {"name": "Bishan", "latitude": (1.3425, 1.3648), "longitude": (103.8322, 103.8584)},
        {"name": "Jurong West", "latitude": (1.3355, 1.3867), "longitude": (103.6785, 103.7267)},
        {"name": "Jurong East", "latitude": (1.3258, 1.3477), "longitude": (103.7065, 103.7610)},
        {"name": "Boon Lay", "latitude": (1.3056, 1.3514), "longitude": (103.6972, 103.7255)},
        {"name": "Woodlands", "latitude": (1.416, 1.466), "longitude": (103.751, 103.803)},
        {"name": "Punggol", "latitude": (1.3983, 1.4282), "longitude": (103.8963, 103.9333)},
        {"name": "Seng Kang", "latitude": (1.3770, 1.4094), "longitude": (103.8753, 103.9027)},
        {"name": "Hougang", "latitude": (1.3615, 1.3847), "longitude": (103.8756, 103.9144)},
        {"name": "Queenstown", "latitude": (1.2804, 1.3043), "longitude": (103.7741, 103.8059)},
        {"name": "Clementi", "latitude": (1.2962, 1.3223), "longitude": (103.7620, 103.7894)},
        {"name": "Orchard Road", "latitude": (1.3014, 1.3055), "longitude": (103.8326, 103.8395)},
        {"name": "Downtown Core", "latitude": (1.2799, 1.2973), "longitude": (103.8521, 103.8621)},
        {"name": "Marina Bay", "latitude": (1.2780, 1.2895), "longitude": (103.8550, 103.8630)}
    ]

    if not any(d["name"] == area_name for d in residential_areas):
        print(f"No residential area found with the name '{area_name}'")
        return None

    area_df = pd.DataFrame(columns=data.columns)

    area = next(d for d in residential_areas if d["name"] == area_name)

    for index, row in data.iterrows():
        latitude = row["latitude"]
        longitude = row["longitude"]

        lat_range = area["latitude"]
        lon_range = area["longitude"]

        if (lat_range[0] <= latitude <= lat_range[1]) and (
            lon_range[0] <= longitude <= lon_range[1]
        ):
            area_df = pd.concat(
                [area_df, pd.DataFrame([row], columns=data.columns)], ignore_index=True
            )

    return area_df

## Extract all the POIs that are within the chosen area.

*Please choose an area name that you wish to test based on the dictionary list of residential areas provided above.*

In [10]:
area_name = "Bishan" #Edit this
area_poi_df = is_within_residential_area(area_name, poi_df)
print(f"Number of POIs within {area_name}: {len(area_poi_df)}")
area_poi_df

C:\Users\fabef\AppData\Local\Temp\ipykernel_97824\1692061522.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  area_df = pd.concat(


Number of POIs within Bishan: 326


,name,latitude,longitude,type
0,101 BISHAN ST 12,1.344859,103.848401,HDB Property
1,102 BISHAN ST 12,1.345143,103.847991,HDB Property
2,103 BISHAN ST 12,1.345512,103.848314,HDB Property
3,104 BISHAN ST 12,1.345675,103.848033,HDB Property
4,105 BISHAN ST 12,1.345895,103.847365,HDB Property
...,...,...,...,...
321,514B BISHAN ST 13,1.350784,103.850396,HDB Property
322,514C BISHAN ST 13,1.350643,103.850394,HDB Property
323,Junction 8,1.350172,103.848554,Shopping Mall
324,Thomson V,1.353195,103.835565,Shopping Mall


## Extract all the Carparks that are within the chosen area

In [11]:
area_cp_df = is_within_residential_area(area_name, cp_df)
print(f"Number of Carparks within {area_name}: {len(area_cp_df)}")
area_cp_df

Number of Carparks within Bishan: 25


C:\Users\fabef\AppData\Local\Temp\ipykernel_97824\1692061522.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  area_df = pd.concat(


,name,latitude,longitude,type
0,AM20,1.364698,103.846089,HDB Carpark
1,AM22,1.364588,103.847653,HDB Carpark
2,AM96,1.364074,103.850940,HDB Carpark
3,BE13,1.344886,103.851528,HDB Carpark
4,BE14,1.347059,103.855432,HDB Carpark
5,BE15,1.345163,103.854838,HDB Carpark
6,BE20,1.348892,103.848665,HDB Carpark
7,BE23,1.350096,103.849763,HDB Carpark
8,BE25,1.355154,103.848309,HDB Carpark
9,BE27,1.356656,103.848631,HDB Carpark


## Create a function where we are able to plot the map of all the carparks (in red) and point of interests (in green) of the chosen area.

Not a must to run this.

In [12]:
def display_map_with_markers(area_name, poi_df, cp_df):
    # Define the center of the map based on the specified area
    center_coordinates = {
        "Tampines": [1.3496, 103.9355],
        "Bishan": [1.3508, 103.8395],
        "Jurong West": [1.3396, 103.7044],
        "Jurong East": [1.3331, 103.7421],
        "Boon Lay": [1.3465, 103.7122],
        "Woodlands": [1.4369, 103.7866],
        "Punggol": [1.4052, 103.9021],
        "Seng Kang": [1.3917, 103.8954],
        "Hougang": [1.3734, 103.8861],
        "Queenstown": [1.2945, 103.7853],
        "Clementi": [1.3151, 103.7653],
        "Orchard Road": [1.3051, 103.8325],
        "Downtown Core": [1.2868, 103.8535],
        "Marina Bay": [1.2829, 103.8547]
    }
    
    # Check if the specified area name exists in the center_coordinates dictionary
    if area_name not in center_coordinates:
        print(f"No coordinates found for the area '{area_name}'")
        return None
    
    # Create a map centered at the specified area
    area_map = folium.Map(location=center_coordinates[area_name], zoom_start=14)

    # Add markers for all the carparks in the specified area (blue markers)
    if cp_df is not None:
        for _, row in cp_df.iterrows():
            folium.Marker([row["latitude"], row["longitude"]], popup=row["name"], icon=folium.Icon(color='red')).add_to(area_map)

    # Add markers for all the POIs in the specified area (green markers)
    if poi_df is not None:
        for _, row in poi_df.iterrows():
            folium.Marker([row["latitude"], row["longitude"]], popup=row["name"], icon=folium.Icon(color='green')).add_to(area_map)

    # Display the map
    return area_map



In [13]:
area_map = display_map_with_markers(area_name, area_poi_df, area_cp_df)
area_map

## Calculate Distance Matrix
**Prompt**

I have 2 prepared datasets.
1. Carparks, with name, latitude, longtitude and type - this has 1273 carparks
2. Points of interest, with name, latitude, longtitude and type - this has 13032 POIs

Construct a D_ij matrix using the haversine formula. 

Due to CPU limitations, the datasets are made smaller based on residential areas that we have chosen earlier.

In [14]:
from math import radians, sin, cos, sqrt, atan2
import pickle
import pandas as pd

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of Earth in kilometers
    return distance * 1000  # Convert to meters

# Function to construct distance matrix and save it to a pickle file
def construct_distance_matrix(bishan_cp_df, bishan_pois_df, filename):
    D_ij = {}

    for cp_index, carpark in bishan_cp_df.iterrows():
        D_ij[cp_index] = {}

        for poi_index, poi in bishan_pois_df.iterrows():
            distance = haversine_distance(
                carpark["latitude"],
                carpark["longitude"],
                poi["latitude"],
                poi["longitude"],
            )

            D_ij[cp_index][poi_index] = distance

    # Save the distance matrix to a pickle file
    with open(filename, "wb") as f:
        pickle.dump(D_ij, f)

    return D_ij

In [17]:
pkl_filename = f"data/pkl/{area_name.lower().replace(' ', '_')}_dm.pkl"
D_ij = construct_distance_matrix(area_cp_df, area_poi_df, pkl_filename)

D_ij

{0: {0: 2220.873084688636,
  1: 2184.6313162083375,
  2: 2147.600099403706,
  3: 2126.24300841733,
  4: 2095.587089965702,
  5: 2077.115840267497,
  6: 2042.8342358565556,
  7: 2004.8840945245004,
  8: 2006.79530315842,
  9: 1934.7393960497195,
  10: 1972.6464247637614,
  11: 2015.890073125863,
  12: 1965.521039055774,
  13: 1931.8985729845288,
  14: 1889.0904430376177,
  15: 1912.8114957348207,
  16: 2013.150948558114,
  17: 2056.4431515149076,
  18: 1991.351567913638,
  19: 2010.8073464718098,
  20: 2023.7200359059168,
  21: 2050.593314062618,
  22: 2082.4392438273057,
  23: 2106.549811990239,
  24: 2046.884392409115,
  25: 2154.245841092004,
  26: 2162.4052378382457,
  27: 2125.7384017742793,
  28: 2166.9594602500156,
  29: 2159.8352072509847,
  30: 2193.734843066095,
  31: 2202.757755251535,
  32: 2228.1264581897162,
  33: 2245.616675000022,
  34: 2312.361566580801,
  35: 2280.5457380788935,
  36: 2371.315377859956,
  37: 2339.5880968031393,
  38: 2318.6066552245825,
  39: 2284.004

# EV Optimal Placement Optimisation
**Prompt**

Below is the problem formulation for the EV optimal placement optimisation problem.

Objective:
$$\text{Max} \sum_{i \in B} \sum_{j \in D} Y_{ij}$$

Decision Variables:
1. **Number of Chargers (Ni)**:
$$N_i \geq P, \forall i \in B$$

2. **Coverage Radius (Ri)**:
$$R_i = k \times N_i, \forall i \in B$$

3. **Distance Constraint**:
$$R_i \leq R_{\text{max}}, \forall i \in B$$

4. **Coverage Constraints (Yij)**:
$$D_{ij} \leq R_i + 99999 \times (1 - Y_{ij}), \forall i \in B, j \in D$$
$$D_{ij} \geq R_i - 99999 \times Y_{ij}, \forall i \in B, j \in D$$
$$\sum_{i \in B} Y_{ij} \geq 1, \forall j \in D$$

5. **Budget Constraint**:
$$\sum_{i \in B} C \times N_i \leq W$$

Assume the role of a data engineer specialized in data and prescriptive analytics. You are proficient in constructing mathematical models and adept at programming and solving these models using the Gurobi library. In our subsequent interactions, you will be tasked with coding various optimization models utilizing the Gurobi library. Your objectives are to ensure accurate results and adhere to the following guidelines:
1. Generate random parameters when specific parameters are not provided.
2. After solving each optimization problem, verify the problem status. If the status is "OPTIMAL", include code snippets to retrieve and display the optimal solutions and the objective value.
3. Disable output logging by setting the "OutputFlag" to 0.
4. Present the results in a concise and clear format, utilizing tables or graphical representations when beneficial.
5. Strive for code efficiency, utilizing the minimal amount of code necessary for model programming.
6. If possible, use “addVars” for non-scalar decision variables and use “addConstrs” for a batch of constraints. Also, please use “quicksum” instead of “sum”.
7. Be careful of the indices given that Python starts from 0 but most of the models start from 1. 
8. Make sure the code you provided fulfills all the requirements, i.e., a complete code should be provided. 

Please define the EV Charger Placement Optimisation problem as a function with detailed comments in the code, solve the problem and retrieve the optimal solutions and objective value, and return the solution.

(Note: Of course, the GAI generated code was modified quite significantly to meet all of above's complex requirements.)

In [18]:
# NEW
def solve_ev_optimal_placement(B, D, P, k, R_max, C, W, D_ij, print_results=True):
    """
    Solve the EV Optimal Placement Optimization problem.

    Parameters:
    - B (int): Number of car parks.
    - D (int): Number of points of interest.
    - P (int): Minimum number of chargers required at each car park.
    - k (float): Constant for coverage radius calculation.
    - R_max (float): Maximum coverage radius.
    - C (float): Variable cost of installing one charging station.
    - W (float): Total budget.
    - D_ij (dict): Dictionary containing distances between car parks and points of interest.

    Returns:
    - model: Gurobi model object.
    """
    # Create a new Gurobi model
    model = gp.Model("EV_Optimal_Placement")

    # Disable output logging
    model.Params.OutputFlag = 0

    # Decision variables
    Ni = model.addVars(B, lb=P, vtype=GRB.INTEGER, name="Ni")  # Number of chargers
    Ri = model.addVars(B, name="Ri")  # Coverage radius
    Y = model.addVars(B, D, vtype=GRB.BINARY, name="Y")  # Coverage indicator

    # Objective function
    obj = gp.quicksum(Y[i, j] for i in range(B) for j in range(D))
    model.setObjective(obj, sense=GRB.MAXIMIZE)

    # Constraints
    # Number of chargers constraint
    model.addConstrs((Ni[i] >= P for i in range(B)), name="charger_num_constr")

    # Coverage radius constraint
    model.addConstrs((Ri[i] == k * Ni[i] for i in range(B)), name="radius_constr")

    # Distance constraint
    model.addConstrs((Ri[i] <= R_max for i in range(B)), name="distance_constrs")

    # Coverage constraints

    for i in range(B):
        for j in range(D):
            # D_ij.get(i, {}) returns an empty dictionary if i is not in D_ij
            # .get(j, None) returns None if j is not found in the dictionary returned by D_ij.get(i, {})
            distance = D_ij.get(i, {}).get(j, None)

            if distance is not None:
                # Define binary data x_ij
                #if Dij <= Rmax, Xij = 1, else Xij = 0
                if distance <= R_max:
                    x_ij = 1
                else:
                    x_ij = 0
                # Add constraint for when D_ij is greater than R_max
                model.addConstr(distance * x_ij <= (Ri[i] + 999999 * (1 - Y[i, j])) * x_ij, name=f"coverage_constrs_1_{i}_{j}")
                # Add constraint for when D_ij is less than or equal to R_max
                model.addConstr(distance * x_ij >= (Ri[i] - 999999 * Y[i, j]) * x_ij, name=f"coverage_constrs_2_{i}_{j}")
                # Add constraint to ensure Y_ij = 0 when x_ij = 0
                model.addConstr(Y[i, j] <= x_ij, name=f"coverage_constrs_3_{i}_{j}")

    # Budget constraint
    model.addConstr(
        (gp.quicksum(C * Ni[i] for i in range(B)) <= W), name="budget_constr"
    )

    # Optimize the model
    model.optimize()

    # Verify problem status
    if model.status == GRB.OPTIMAL:
        # Retrieve and display optimal solutions
        df_charger_placement = pd.DataFrame(
            columns=["Carpark", "Number of chargers", "Coverage radius"]
        )
        # Charger placement at each car park
        print("Charger Placement Results:")
        for i in range(B):
            if print_results == True:
                print(
                    f"Carpark {i}: Number of chargers = {Ni[i].x:.0f}, Coverage radius = {Ri[i].x:.2f} m"
                )
            # make above into a df
            df_charger_placement.loc[i] = [i, Ni[i].x, Ri[i].x]

        # Points of interest coverage
        # print("\nPoints of Interest Coverage:")
        # for j in range(D):
        #    covered = any(Y[i, j].x for i in range(B))
        #    print(f"Point of interest {j}: {'Covered' if covered else 'Not Covered'}")

        # Display objective value
        sum_coverage_radius = sum(Ri[i].x for i in range(B))
        total_pois_covered = sum(
            1 for j in range(D) if any(model.getVarByName(f"Y[{i},{j}]").x for i in range(B))
        )
        proportion_of_pois_covered = (total_pois_covered / D) * 100

        print(f"Sum of coverage radius: {sum_coverage_radius:.2f}")
        print(f"Total POIs covered: {total_pois_covered}")
        print(
            f"Proportion of points of interest covered: {proportion_of_pois_covered:.2f}%\n"
        )

        # Retrieve and display optimal solutions
        obj_value = model.objVal
        return (
            model,
            obj_value,
            sum_coverage_radius,
            proportion_of_pois_covered,
            df_charger_placement,
        )
    else:
        print("No optimal solution found.")
        return None, None, None, None, None

In [20]:
# area_name = "Bishan" #only need this if already have the pkl file otherwise need run from cell 1.
# Load the distance matrix from the pickle file
pkl_filename = f"data/pkl/{area_name.lower().replace(' ', '_')}_dm.pkl"
with open(pkl_filename, "rb") as wl:
    D_ij = pickle.load(wl)

D_ij

{0: {0: 2220.873084688636,
  1: 2184.6313162083375,
  2: 2147.600099403706,
  3: 2126.24300841733,
  4: 2095.587089965702,
  5: 2077.115840267497,
  6: 2042.8342358565556,
  7: 2004.8840945245004,
  8: 2006.79530315842,
  9: 1934.7393960497195,
  10: 1972.6464247637614,
  11: 2015.890073125863,
  12: 1965.521039055774,
  13: 1931.8985729845288,
  14: 1889.0904430376177,
  15: 1912.8114957348207,
  16: 2013.150948558114,
  17: 2056.4431515149076,
  18: 1991.351567913638,
  19: 2010.8073464718098,
  20: 2023.7200359059168,
  21: 2050.593314062618,
  22: 2082.4392438273057,
  23: 2106.549811990239,
  24: 2046.884392409115,
  25: 2154.245841092004,
  26: 2162.4052378382457,
  27: 2125.7384017742793,
  28: 2166.9594602500156,
  29: 2159.8352072509847,
  30: 2193.734843066095,
  31: 2202.757755251535,
  32: 2228.1264581897162,
  33: 2245.616675000022,
  34: 2312.361566580801,
  35: 2280.5457380788935,
  36: 2371.315377859956,
  37: 2339.5880968031393,
  38: 2318.6066552245825,
  39: 2284.004

## Running to get the optimal number of chargers

In [21]:
# ACTUAL

B = len(area_cp_df)  # Number of car parks
D = len(area_poi_df)  # Number of points of interest 
P = 3  # Minimum number of chargers required at each car park
R_max = 550  # Maximum coverage radius (in meters)
k = 50  # Constant for coverage radius calculation (in meters)

C = 10500  # Variable cost of installing one charging station
W = ((B/1273)*60000)*10500  # Total budget for area = [(number of carparks at area/1273)*60000]*10500 =

# Solve the EV Optimal Placement Optimization problem
model, obj_value, sum_coverage_radius, proportion_of_pois_covered, df_charger_placement = solve_ev_optimal_placement(B, D, P, k, R_max, C, W, D_ij)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-04


Charger Placement Results:
Carpark 0: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 1: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 2: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 3: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 4: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 5: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 6: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 7: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 8: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 9: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 10: Number of chargers = 8, Coverage radius = 400.00 m
Carpark 11: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 12: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 13: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 14: Number of chargers = 11, Coverage radius = 550.00 m
Carpark 15: Number of ch

In [22]:
# Results
print("Objective value:", obj_value)
print("Total coverage radius:", sum_coverage_radius)
print("\nCharger Placement Results:")
# value counts of number of chargers
print(df_charger_placement['Number of chargers'].value_counts())
# value counts of coverage radius
print(df_charger_placement['Coverage radius'].value_counts())
# value counts of number of chargers and coverage radius together
print(df_charger_placement.groupby(['Number of chargers', 'Coverage radius']).size())

df_charger_placement

Objective value: 1584.0
Total coverage radius: 13450.0

Charger Placement Results:
Number of chargers
11.0    23
8.0      2
Name: count, dtype: int64
Coverage radius
550.0    23
400.0     2
Name: count, dtype: int64
Number of chargers  Coverage radius
8.0                 400.0               2
11.0                550.0              23
dtype: int64


,Carpark,Number of chargers,Coverage radius
0,0.0,11.0,550.0
1,1.0,11.0,550.0
2,2.0,11.0,550.0
3,3.0,11.0,550.0
4,4.0,11.0,550.0
5,5.0,11.0,550.0
6,6.0,11.0,550.0
7,7.0,11.0,550.0
8,8.0,11.0,550.0
9,9.0,11.0,550.0


## Function to plot the map with all the carparks (red) and point of interests (green) and the coverage radius from each carparks.

In [23]:
def plot_optimal_charger_placement(model, area_cp_df, area_pois_df, area_name):
    # Define the center of the map based on the specified area
    center_coordinates = {
        "Tampines": [1.3496, 103.9355],
        "Bishan": [1.3508, 103.8395],
        "Jurong West": [1.3396, 103.7044],
        "Jurong East": [1.3331, 103.7421],
        "Boon Lay": [1.3465, 103.7122],
        "Woodlands": [1.4369, 103.7866],
        "Punggol": [1.4052, 103.9021],
        "Seng Kang": [1.3917, 103.8954],
        "Hougang": [1.3734, 103.8861],
        "Queenstown": [1.2945, 103.7853],
        "Clementi": [1.3151, 103.7653],
        "Orchard Road": [1.3051, 103.8325],
        "Downtown Core": [1.2868, 103.8535],
        "Marina Bay": [1.2829, 103.8547]
    }
    
    # Check if the specified area name exists in the center_coordinates dictionary
    if area_name not in center_coordinates:
        print(f"No coordinates found for the area '{area_name}'")
        return None
    
    # Create a map centered at the specified area
    results_map = folium.Map(location=center_coordinates[area_name], zoom_start=14)

    # Add markers for all the POIs in the specified area 
    for _, row in area_pois_df.iterrows():
        if row["type"] == "HDB Property":
            # Green markers for HDB property
            folium.Marker([row["latitude"], row["longitude"]], popup=row["name"], icon=folium.Icon(color='green')).add_to(results_map)
        elif row["type"] == "Shopping Mall":
            # Dark green markers for shopping mall
            folium.Marker([row["latitude"], row["longitude"]], popup=row["name"], icon=folium.Icon(color='darkgreen')).add_to(results_map)

    # Add markers for the optimal charger placement at each car park (red markers), otherwise blue markers
    for i, carpark in area_cp_df.iterrows():
        carpark_name = carpark["name"]
        if model.getVarByName(f"Ni[{i}]").x > 0:
            num_charger = model.getVarByName(f"Ni[{i}]").x
            popup_html = f"<b>Car Park:</b> {carpark_name}<br><b>Chargers:</b> {num_charger:.0f}"
            folium.Marker([carpark["latitude"], carpark["longitude"]], popup=popup_html, icon=folium.Icon(color='red')).add_to(results_map)

            # Show radius coverage of car park with chargers
            radius = model.getVarByName(f"Ri[{i}]").x
            folium.Circle(location=[carpark["latitude"], carpark["longitude"]], radius=radius, color='red', fill=True, fill_color='red', fill_opacity=0.2).add_to(results_map)
        else:
            num_charger = 0
            popup_html = f"<b>Car Park:</b> {carpark_name}<br><b>Chargers:</b> {num_charger:.0f}"
            folium.Marker([carpark["latitude"], carpark["longitude"]], popup=popup_html, icon=folium.Icon(color='blue')).add_to(results_map)

    return results_map


In [21]:
results_map = plot_optimal_charger_placement(model, area_cp_df, area_poi_df, area_name)
results_map